# plex

------
current status: **INCOMPLETE**

- goal: install and configure a plex server
- details:
  - single ec2 server running a plex instance: https://www.plex.tv/
  - behind elb
  - ASG
  - s3fs-fuse https://github.com/s3fs-fuse/s3fs-fuse

In [51]:
from plex_settings import domain_name, pms_rpm_url
from troposphere import Template, Ref, GetAtt, GetAZs, Base64, Join, FindInMap, Select, Output, Parameter, Tags
import troposphere.ec2 as ec2
import troposphere.elasticloadbalancing as elb
import boto3

In [26]:
resources = set()
t = Template()
t.add_version('2010-09-09')
t.add_description('plex behind an elb and ASG')

# common tag to apply to most resources
domain_name_tags = Tags(Site=domain_name)

# loadbalance across 2 AZs
az1 = Select('0', GetAZs(''))
az2 = Select('1', GetAZs(''))
az3 = Select('2', GetAZs(''))

In [50]:
plex_vpc = ec2.VPC('plex', CidrBlock='10.0.0.0/16', Tags=domain_name_tags)
resources.add(plex_vpc);

{'CidrBlock': (str, True),
 'EnableDnsHostnames': (<function troposphere.validators.boolean>, False),
 'EnableDnsSupport': (<function troposphere.validators.boolean>, False),
 'InstanceTenancy': (str, False),
 'Tags': (list, False)}

In [28]:
web_subnet = ec2.Subnet('webserver',
                        AvailabilityZone=az1,
                        CidrBlock='10.0.1.0/24',
                        VpcId=Ref(plex_vpc),
                        Tags=domain_name_tags)
resources.add(web_subnet);

In [59]:

web_elb = elb.LoadBalancer('webElb',
                           AvailabilityZones=az1,
                           Subnets=Ref(web_subnet),
                           Tags=domain_name_tags)


AttributeError: 'function' object has no attribute 'props'

In [29]:
file_name = 'plex.json'

def build(filename):
    fname = 'outputs/' + filename
    for r in resources:
        t.add_resource(r)
    
    with open(fname, 'w') as fd:
        fd.write(t.to_json())
    
    !aws cloudformation validate-template --template-body file://{fname} > /dev/null && echo "build ok"

build(file_name)

build ok


In [31]:
client = boto3.client('cloudformation')

In [35]:
client.create_stack(StackName='plex',
                    TemplateBody=t.to_json(),
                    Parameters=[],
                    Tags=domain_name_tags.to_dict())

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '374',
   'content-type': 'text/xml',
   'date': 'Tue, 07 Feb 2017 02:41:27 GMT',
   'x-amzn-requestid': 'ec7f2890-ecde-11e6-9e6c-d52f29006dc2'},
  'HTTPStatusCode': 200,
  'RequestId': 'ec7f2890-ecde-11e6-9e6c-d52f29006dc2',
  'RetryAttempts': 0},
 'StackId': 'arn:aws:cloudformation:us-west-2:696898316521:stack/plex/ec878ca0-ecde-11e6-9994-503aca41a029'}

In [49]:
for event in client.describe_stack_events(StackName='plex')['StackEvents']:
    print(event['Timestamp'], event['ResourceStatus'], event['ResourceType'])

ClientError: An error occurred (ValidationError) when calling the DescribeStackEvents operation: Stack [plex] does not exist

In [43]:
client.delete_stack(StackName='plex')

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '212',
   'content-type': 'text/xml',
   'date': 'Tue, 07 Feb 2017 02:47:32 GMT',
   'x-amzn-requestid': 'c632f8a0-ecdf-11e6-aeb8-3b3a524a1df0'},
  'HTTPStatusCode': 200,
  'RequestId': 'c632f8a0-ecdf-11e6-aeb8-3b3a524a1df0',
  'RetryAttempts': 0}}